💻 Importer les **bibliothèques nécessaires** à l'activité

In [ ]:
import requests
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## 🎣 Récupérer un produit de la base de données d'Open Food Facts

💻 Récupérer **500 produits** de la **catégorie "snacks"**

In [ ]:
def get_open_food_facts_data(category, page_size=100, num_pages=5):
    base_url = "https://world.openfoodfacts.org/cgi/search.pl"
    products = []
    for page in range(1, num_pages + 1):
        params = {
            'action': 'process',
            'tagtype_0': 'categories',
            'tag_contains_0': 'contains',
            'tag_0': category,
            'page_size': page_size,
            'page': page,
            'json': 1
        }
        response = requests.get(base_url, params=params)
        data = response.json()
        products.extend(data.get('products', []))
    return products

category = 'snacks'
data = get_open_food_facts_data(category)
df = pd.json_normalize(data)
df

,_id,_keywords,added_countries_tags,additives_debug_tags,additives_n,additives_old_n,additives_old_tags,additives_original_tags,additives_prev_original_tags,additives_tags,...,images.ingredients_yi.sizes.full.h,images.ingredients_yi.sizes.full.w,images.ingredients_yi.white_magic,images.ingredients_yi.x1,images.ingredients_yi.x2,images.ingredients_yi.y1,images.ingredients_yi.y2,languages.en:yiddish,languages_codes.yi,ecoscore_extended_data.impact.likeliest_recipe.fr:lai___cr_m__en_poudre
0,7622210449283,"[35, and, au, biscuit, ble, cake, charte, choc...",[],"[en-e322i-added, en-e450-added, en-e500ii-adde...",4.0,4.0,"[en:e502, en:e450, en:e1403, en:e322]","[en:e503ii, en:e500ii, en:e450i, en:e322i]","[en:e503, en:e500, en:e450i, en:e322]","[en:e322, en:e322i, en:e450, en:e450i, en:e500...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3175680011480,"[arome, au, biscuit, colorant, de, en, et, eur...",[],[],4.0,6.0,"[en:e1403, en:e322, en:e336, en:e500, en:e503,...","[en:e322i, en:e336, en:e500, en:e503]","[en:e341, en:e504, en:e322, en:e336, en:e500, ...","[en:e322, en:e322i, en:e336, en:e500, en:e503]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3046920022651,"[70, and, bar, cacao, chocolate, cocoa, dark, ...",[],[],0.0,0.0,[],[],[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3046920029759,"[90, and, chocolate, cocoa, dark, excellence, ...",[],NaN,0.0,NaN,NaN,[],NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3046920022606,"[85, agriculture, and, bar, belgium, cacao, ch...",[],[],0.0,0.0,[],[],[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,3497911119124,"[and, appetizer, beverage, bret, brittany, cer...",[],[],0.0,0.0,[],[],[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,4000607852008,"[barre, cacao, chocolat, chocolatee, confiseri...",[],[],1.0,0.0,[],[en:e322],[],[en:e322],...,393.0,2215.0,false,307.99595382462684,2522.9690881529846,1757.4718341884327,2150.4509386660447,2.0,2.0,NaN
497,3229820800325,"[ab, agriculture, and, biologique, biscuit, bj...",[],NaN,4.0,4.0,"[en:e503, en:e500, en:e1403, en:e414]","[en:e503, en:e500, en:e414, en:e392]",NaN,"[en:e392, en:e414, en:e500, en:e503]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,7622201685003,"[appetizer, colorant, conservateur, cracker, r...",[],NaN,3.0,4.0,"[en:e341, en:e503, en:e500, en:e501]","[en:e341, en:e503, en:e500]",NaN,"[en:e341, en:e500, en:e503]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


💻 Analyser les **valeurs manquantes**

💻 Sélectionner les **colonnes les plus intéressantes** (5 à 10 colonnes)

In [ ]:
# Sélectionner les colonnes d'intérêt
columns = ['product_name', 'nutriments.energy_100g', 'nutriments.fat_100g',
           'nutriments.carbohydrates_100g', 'nutriments.proteins_100g', 'nutriments.sugars_100g',
           'nutriments.salt_100g']
df = df[columns]
# Renommer les colonnes pour simplifier
df.columns = ['product_name', 'energy', 'fat', 'carbohydrates', 'proteins', 'sugars', 'salt']

💻 Traiter les **valeurs manquantes**

In [ ]:
df = df.dropna()

💻 **Normaliser** les données

In [ ]:
# Normaliser les données
nutritional_data = df[['energy', 'fat', 'carbohydrates', 'proteins', 'sugars', 'salt']]
normalized_data = (nutritional_data - nutritional_data.mean()) / nutritional_data.std()

💻 Calculer la **matrice de similarité cosinus**

In [ ]:
# Calculer la similarité cosinus
similarity_matrix = cosine_similarity(normalized_data)

💻 Créer le **moteur de recommandation**

In [ ]:
def recommend_products(product_name, df, similarity_matrix, top_n=5):
    matching_products = df[df['product_name'].str.contains(product_name, case=False, na=False)]
    if matching_products.empty:
        print(f"Produit '{product_name}' non trouvé.")
        return pd.DataFrame()
    product_index = matching_products.index[0]
    similarity_scores = list(enumerate(similarity_matrix[product_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similar_products_indices = [i[0] for i in similarity_scores[1:top_n + 1]]
    return df.iloc[similar_products_indices][['product_name', 'energy', 'fat', 'carbohydrates', 'proteins', 'sugars', 'salt']]

💻 Tester le moteur de recommandation sur un **exemple de produit**

In [ ]:
# Exemple de recommandation
product_name = "Crackers"
recommended_products = recommend_products(product_name, df, similarity_matrix, top_n=5)
print(recommended_products)

                              product_name  energy   fat  carbohydrates  \
366       BelVita - Petit déjeuner Tartiné  1926.0  16.0           69.0   
22                  Biscuits goût chocolat  1944.0  17.0           69.0   
355  Duo Fourré Goût Chocolat 🍫 Noisette 🥜  1880.0  15.0           67.2   
497         Biscuits fourrés chocolat noir  2004.0  20.0           65.0   
25                 Fourrés - Chocolat Noir  2017.0  20.0           65.0   

     proteins  sugars   salt  
366      7.40    26.0  0.560  
22       6.50    30.0  0.470  
355      7.51    27.7  0.514  
497      7.10    27.0  0.500  
25       7.60    27.0  0.490  
